In [ ]:
# import tensorflow
import tensorflow as tf
import zipfile, os, splitfolders
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Mengekstrak File Zip Dataset Machine Learning
lokal_zip = '/tmp/rockpaperscissors.zip'
zip_ext = zipfile.ZipFile(lokal_zip, 'r')
zip_ext.extractall('/tmp')
zip_ext.close()

# Membagi isi direktori dataset menjadi 2 folder train dan val menggunakan split folders
# Menggunakan rasio 60 : 40 dari dataset yang tersedia
splitfolders.ratio('/tmp/rockpaperscissors/rps-cv-images', '/tmp/rockpaperscissors/dataset', seed=1, ratio=(.6, .4))

In [ ]:
# Mendefinisikan variabel direktori train dan validasi
base_dir = '/tmp/rockpaperscissors/dataset'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
validation_dir
os.listdir(train_dir)
os.listdir(validation_dir)

In [ ]:
# Mengecek Jumlah Data Sample pada direktori train
list_train = os.listdir(train_dir)
jumlah = 0

for i in range(len(list_train)):
  data = len(os.listdir(os.path.join(train_dir,list_train[i])))
  jumlah += data
  print(f"Data {list_train[i]} = {data}")
else:
  print(f"Jumlah data train {jumlah}\n")

# Mengecek Jumlah Data Sample pada direktori validasi
list_val = os.listdir(validation_dir)
jumlah = 0

for i in range(len(list_val)):
  data = len(os.listdir(os.path.join(validation_dir,list_val[i])))
  jumlah += data
  print(f"Data {list_val[i]} = {data}")
else:
  print(f"Jumlah data validasi {jumlah}")

# Karena 60% dari 2188 sample data adalah 1312.8
# Maka Python membulatkan sample datanya menjadi 1312 sample data

# dan sisanya masuk ke data validasinya

In [ ]:
# Mendefinisakan variabel direktori train rps dan val rps secara terpisah
train_rock_dir = os.path.join(train_dir, 'train')
train_scissors_dir = os.path.join(train_dir, 'scissors')
train_paper_dir = os.path.join(train_dir, 'paper')

validation_crock_dir = os.path.join(validation_dir, 'rock')
validation_scissors_dir = os.path.join(validation_dir, 'scissors')
validation_paper_dir = os.path.join(validation_dir, 'paper')

In [ ]:
# Augmentasi Gambar Menggunakan Image Data Generator dari library keras
train_datagen = ImageDataGenerator(
  rescale=1./255,
  rotation_range=0.2,
  horizontal_flip=True,
  shear_range=0.2,
  fill_mode = "nearest")

test_datagen = ImageDataGenerator(
  rescale=1./255,
  rotation_range=0.2,
  horizontal_flip=True,
  shear_range=0.2,
  fill_mode = "nearest")

In [ ]:
# Membuat flow datanya darimana dataset kita berasal
train_generator = train_datagen.flow_from_directory(
  train_dir,
  target_size=(150, 150),
  batch_size=32,
  class_mode='categorical')
  
validation_generator = test_datagen.flow_from_directory(
  validation_dir,
  target_size=(150, 150),
  batch_size=32,
  class_mode='categorical')

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, 
          steps_per_epoch=25, 
          epochs=20, 
          validation_data=validation_generator,
          validation_steps=5,
          verbose=2)

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
 
uploaded = files.upload()
 
for fn in uploaded.keys():
 
  # predicting images
  path = fn
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
 
  images = np.vstack([x])
  classes = model.predict(images, batch_size=32)

  if classes[0,0]!=0:
    print('PAPER')
  elif classes[0,1]!=0:
    print('ROCK')
  else:
    print('SCISSORS')